In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import duckdb
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
warnings.filterwarnings('once')

### Notebook Goal
> This notebook aims to be a first draft of algorithm to detech fraudulent transactions

- Data Historic:
  - sources: transactions & transactions_label  
  - ds_1: denormalized_table joinning transactions & transactions_label
  - ds_2: cleanned data to only keep rows where label <=1 and drop column amount
  - ds_3: add new column that describes if a user did a fraudulent transaction in the past
  - ds_4: Cast created_at column in int

In [ ]:
"""
Transactions Tables
Describes every transactions performed
"""
transactions = duckdb.sql("SELECT * FROM 'data/transactions.csv'")
transactions.describe()

In [ ]:
"""
Transactions Labels Tables
Describes for every transactions if it is fraudulent (1) or not (0)
"""
transactions_label = duckdb.sql("SELECT * FROM 'data/transactions_label.csv'")
transactions_label.describe()

### Transformation | Denormalization
**Exercice:** Using an SQL query, construct a new table that allows containing all elements from the **transactions** table with the associated label (denormalization).

In [ ]:
input_query = "!!!!TO OVERRIDE!!!!"

ds_1 = duckdb.sql(input_query)
ds_1.describe()

### Transformation | Cleaning
**Exercise:** Using an SQL query, construct a new table that keeps only the labels <= 1.

In [ ]:
input_query = "!!!!TO OVERRIDE!!!!"

ds_2 = duckdb.sql(input_query)
ds_2.describe()

### Transformation | Feature Engineering
**Exercice:** We assume that if a user has made a fraudulent transaction in the past, there is a high probability that they will make new fraudulent transactions. Therefore, we want to add a new column, user_fraudulent, to the previous table using an SQL query. This column returns 1 if the user has made a fraudulent transaction in the past, and 0 otherwise. Tips: We can break down the problem into two sub-problems:

1. Calculate the cumulative sum of the number of fraudulent transactions for each user over time.
2. If the cumulative sum > 0, then the user is suspicious.

In [ ]:
input_query = "!!!!TO OVERRIDE!!!!"

ds_3 = duckdb.sql(input_query)
ds_3.describe()

### Transformation | Type Casting

In [ ]:
input_query = "SELECT user_id, transaction_id, user_fraudulent, epoch_ms(created_at) as created_at, label \
                   FROM ds_3"

ds_4 = duckdb.sql(input_query)
ds_4.describe()

### Training | Split Train/Test

In [ ]:
train, test = train_test_split(ds_4.df(), test_size=0.30)
train_x = train.loc[:, ['user_id', 'transaction_id', 'created_at', 'user_fraudulent']]
train_y = train.loc[:, ['label']]
test_x = test.loc[:, ['user_id', 'transaction_id', 'created_at', 'user_fraudulent']]
test_y = test.loc[:, ['label']]
print(f"train_size: {train_x.size}, test_size: {test_x.size}")

### Training | Plotting labels distribution

In [ ]:
print(f"Train_x : {train_x.shape}, Test_x : {test_x.shape}\nTrain_y : {train_y.shape}, Test_y : {test_y.shape}")
figure = plt.figure(figsize=(8, 4))
ax = plt.subplot(121)
train_y['label'].hist()
plt.title("Train Label Distribution")
ax.grid(True)
ax = plt.subplot(122)
test_y['label'].hist()
plt.title("Test Label Distribution")
ax.grid(True)

### Training | Prepare Pipeline
**Exercice:** Modify the following configuration to test a new classifier: **[DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)** with the hyperparameter **criterion** varying in the set **("gini", "entropy", "log_loss")**

In [ ]:
#Build all pipelines
all_models_name = ["Logistic Regression", "DecisionTreeClassifier"]
all_models_parameters = []
all_models_parameters.append(
{
    'pipeline' : Pipeline([
        ('Standardization', StandardScaler()),
        ('LogisticRegression', LogisticRegression())
    ]),
    'parameters' : [
        {
            'LogisticRegression__max_iter' : range(40, 50)
        }
    ],
    "train_x" : train_x,
    "train_y" : train_y,
    "test_x" : test_x,
    "test_y" : test_y,
    
    "scores_names" : ["Accuracy"], 
    "scores_obj" : [accuracy_score]  

})

In [ ]:
def return_best_optimized_model(pipeline, parameters, train_x, train_y, test_x, test_y, scores_names, scores_obj):
    gridsearch = GridSearchCV(estimator=pipeline, param_grid=parameters).fit(train_x, train_y.values.ravel())
    evaluation_scores = return_evaluation_scores(gridsearch, scores_names, scores_obj, test_x, test_y.values.ravel())
    return gridsearch, evaluation_scores

def return_evaluation_scores(current_pipeline, scores_names, scores_obj, test_x, test_y):
    results = {}
    for name, obj in zip(scores_names, scores_obj):
        results[name] = obj(current_pipeline.predict(test_x), test_y)
    return results


In [ ]:
all_bests_models = []
#Find bests Hyperparams for each models
for name, model in zip(all_models_name, all_models_parameters):
    print(f"Finding best hyper parameters for model:{name}")
    all_bests_models.append(return_best_optimized_model(**model))
print(f"Models with its best hyper-parameters and associated test evaluations: {all_bests_models}")    